# Data Preparation

A manual files inspection is performed before hand to standardize csv files and we would be able to load the content in appropate dataframes as shown in next.
Once we are able to load files to dataframe we investigate data integraty on the content level.

In [1]:
import pandas as pd
import numpy as np

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

### Load *.csv

In [2]:
df_demographics = pd.read_csv("../specs/Model Build - Demographics.csv", encoding='latin-1', index_col=False)
df_previous_load_holdings = pd.read_csv("../specs/Model Build - Previous Loan Holdings.csv", encoding='latin-1', index_col=False)
df_product_held_in_bank = pd.read_csv("../specs/Model Build - Product Held in Bank.csv", encoding='latin-1', index_col=False)
df_avg_txn_amount = pd.read_csv("../specs/Model Build - TXN Amount.csv", encoding='latin-1', index_col=False)
df_target_loan_flag = pd.read_csv("../specs/Target Variable - Purchased Loan Flag.csv", encoding='latin-1', index_col=False)
df_ca_txn = pd.read_csv("../specs/TEST - Transactions out of Current Account.csv", encoding='latin-1', index_col=False)

### Investigate duplicate customers records, if any

In [3]:
def print_duplicate_status(df, df_name):
    s = df.ClientID.value_counts()
    dup_list = s[s > 1].index
    if dup_list is []:
        print(df_name + " does not contains duplicate records")
    else:
        print(df_name + " contains duplicate records")
        print(df.loc[df['ClientID'].isin(dup_list)])
        
    print("---------------------------------------------------------------------------------------")

print_duplicate_status(df_demographics, "Model Build - Demographics.csv")
print_duplicate_status(df_previous_load_holdings, "Model Build - Previous Loan Holdings.csv")
print_duplicate_status(df_product_held_in_bank, "Model Build - Product Held in Bank.csv")
print_duplicate_status(df_avg_txn_amount, "Model Build - TXN Amount.csv")
print_duplicate_status(df_target_loan_flag, "Target Variable - Purchased Loan Flag.csv")
print_duplicate_status(df_ca_txn, "TEST - Transactions out of Current Account.csv")


Model Build - Demographics.csv contains duplicate records
      ClientID  Age Gender     County    IncomeGroup
45          46   56      0      Kerry  10001 - 40000
46          46   20      0      Laois  10001 - 40000
1220      1220   67      0  Tipperary  10001 - 40000
1221      1220   48      1    Kildare        100000+
3675      3674   46      0     Galway  10001 - 40000
3676      3674   37      1     Carlow  10001 - 40000
3677      3675   56      1     Dublin      0 - 10000
3678      3675   23      1       Cork  10001 - 40000
---------------------------------------------------------------------------------------
Model Build - Previous Loan Holdings.csv contains duplicate records
      ClientID HeldLoanPreviously
20          21                  1
21          21                  0
46          46                  0
47          46                  0
1221      1220                  0
1222      1220                  0
3676      3674                  0
3677      3675                  0
367

## Conclusion:
As the union set of duplicate client rows reveals a 5 duplicate records out of 10,000, which represents 0.05% of the dataset. We can safely neglect it from further analysis
Otherwise, a deeper investigation of the ETL process of how the dataset is constructed to determine the approparate policy for handling duplicate and/or missing records.

### Remove duplicates

In [4]:
def extract_duplicate_status(df):
    s = df.ClientID.value_counts()
    return s[s > 1].index

dup_client_ids = []
dup_client_ids.extend(extract_duplicate_status(df_demographics))
dup_client_ids.extend(extract_duplicate_status(df_previous_load_holdings))
dup_client_ids.extend(extract_duplicate_status(df_product_held_in_bank))
dup_client_ids.extend(extract_duplicate_status(df_avg_txn_amount))
dup_client_ids.extend(extract_duplicate_status(df_ca_txn))
dup_client_ids = set(dup_client_ids)
print("ClientIDs is be excluded from data set: " + str(dup_client_ids))


ClientIDs is be excluded from data set: {1220, 46, 21, 3674, 3675}


In [5]:
df_demographics = df_demographics[~df_demographics.ClientID.isin(dup_client_ids)]
df_previous_load_holdings = df_previous_load_holdings[~df_previous_load_holdings.ClientID.isin(dup_client_ids)]
df_product_held_in_bank = df_product_held_in_bank[~df_product_held_in_bank.ClientID.isin(dup_client_ids)]
df_avg_txn_amount = df_avg_txn_amount[~df_avg_txn_amount.ClientID.isin(dup_client_ids)]
df_target_loan_flag = df_target_loan_flag[~df_target_loan_flag.ClientID.isin(dup_client_ids)]
df_ca_txn = df_ca_txn[~df_ca_txn.ClientID.isin(dup_client_ids)]


### Persist the clean dataset

In [6]:
pd.DataFrame.to_csv(df_demographics, "../specs/clean/Model Build - Demographics.csv", encoding='utf-8', index=False)
pd.DataFrame.to_csv(df_previous_load_holdings, "../specs/clean/Model Build - Previous Loan Holdings.csv", encoding='utf-8', index=False)
pd.DataFrame.to_csv(df_product_held_in_bank, "../specs/clean/Model Build - Product Held in Bank.csv", encoding='utf-8', index=False)
pd.DataFrame.to_csv(df_avg_txn_amount, "../specs/clean/Model Build - TXN Amount.csv", encoding='utf-8', index=False)
pd.DataFrame.to_csv(df_target_loan_flag, "../specs/clean/Target Variable - Purchased Loan Flag.csv", encoding='utf-8', index=False)
pd.DataFrame.to_csv(df_ca_txn, "../specs/clean/TEST - Transactions out of Current Account.csv", encoding='utf-8', index=False)

### Merge *.csv into Abstract Base Table (ABT)

In [7]:
# features
df_abt = pd.merge(left=df_demographics, right=df_previous_load_holdings, how='left', left_on='ClientID', right_on='ClientID')
df_abt = pd.merge(left=df_abt, right=df_product_held_in_bank, how='left', left_on='ClientID', right_on='ClientID')
df_abt = pd.merge(left=df_abt, right=df_avg_txn_amount, how='left', left_on='ClientID', right_on='ClientID')
df_abt = pd.merge(left=df_abt, right=df_ca_txn, how='left', left_on='ClientID', right_on='ClientID')
df_abt = df_abt.iloc[:, :-4]
# target
df_abt = pd.merge(left=df_abt, right=df_target_loan_flag, how='left', left_on='ClientID', right_on='ClientID')
df_abt.head(5)
df_abt.tail(5)


,ClientID,Age,Gender,County,IncomeGroup,HeldLoanPreviously,NumberOfProductsInbank,AverageTXNAmount,NumTransactions,LastTXNAmount,MerchantCode,LastTransactionNarrative,LoanFlag
0,1,36,1,Cork,10001 - 40000,1,4,58.000,0.0,NaN,NaN,NaN,0
1,2,43,1,Cavan,0 - 10000,0,4,2.663,17.0,83.66,7211.0,THE BRIDGE LAUNDRY WICKLOW TOWN,0
2,3,32,0,Dublin,10001 - 40000,0,2,46.000,25.0,526.18,3667.0,LUXOR HOTEL/CASINO LAS VEGAS NV,0
3,4,52,1,Louth,40001 - 60000,1,2,0.000,13.0,70.68,5712.0,HARVEY NORMAN CARRICKMINES,0
4,5,63,0,Kilkenny,60001 - 100000,0,1,126.000,39.0,259.07,5999.0,PAYPAL *PETEWOODWAR 35314369001,0


,ClientID,Age,Gender,County,IncomeGroup,HeldLoanPreviously,NumberOfProductsInbank,AverageTXNAmount,NumTransactions,LastTXNAmount,MerchantCode,LastTransactionNarrative,LoanFlag
9990,9996,70,1,Cork,10001 ? 40000,0,3,1.501,18.0,332.96,7379.0,LAPTECK LTD GALWAY,0
9991,9997,36,1,Galway,10001 - 40000,0,5,185.000,5.0,222.59,3730.0,MGM GRAND - FRONT DESK LAS VEGAS NV,0
9992,9998,43,1,Laois,10001 - 40000,0,1,0.000,0.0,NaN,NaN,NaN,0
9993,9999,32,0,Dublin,10001 - 40000,1,2,0.000,37.0,853.35,3779.0,W DOHA HOTELS & RESIDENCEDOHA,0
9994,10000,52,1,Dublin,100000+,0,5,0.000,6.0,618.59,3825.0,HOTEL XALET BESOLI ARINSAL,0


As we expected we have *10000-5=9995* client information

## Persist ABT for future analysis

In [8]:
pd.DataFrame.to_csv(df_abt, "../specs/clean/Model Build - AbastractBaseTable.csv", encoding='utf-8', index=False)